In [ ]:
from fastai.text import *
from fastai.text.models import AWD_LSTM

path = Path(".")

In [ ]:
df = pd.read_csv("./Tweets.csv")

In [ ]:
df.head()


In [ ]:
train_df, valid_df = df.loc[:12000,:], df.loc[12000:,:]

In [ ]:
df.shape

In [ ]:
data_lm = TextLMDataBunch.from_df(".", train_df, valid_df, text_cols=10, bs=32)

In [ ]:
data_clas = TextClasDataBunch.from_df(".", train_df, valid_df, text_cols=10, label_cols=1, bs=32)

In [ ]:
df.columns

In [ ]:
data_lm.show_batch()

In [ ]:
data_clas.show_batch()

In [ ]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 1e-02)

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, 1e-02)


In [ ]:
# save encoder
learn.save_encoder('twitter-sentiment-enc')

In [ ]:
# create model and load in encoder
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.3)
learn.load_encoder('twitter-sentiment-enc')

# find and plot learning rate
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 1e-2)

# unfreeze one layer group and train another epoch
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

In [ ]:
learn.predict("I hate flying")